In [ ]:
# Based on https://github.com/mvberg/ib-gateway-docker
# It works pretty well but something you need to restart your kernel (IB gateway keeps running) or kernel dies unexpectedly.
#
# Hacked dirty with hardcoded things but gets you running quick... will probably become outdated eventually.
# Ideally, move to IBC instad of IBController and handle new IB Gateway version automatically
# I cut fragments out so might miss some imports or something but you get an idea

import inspect
import os
import time

try:
    import ib_insync as ibi
    ibi.util.startLoop()  # when in a notebook
except ModuleNotFoundError:
    !pip -qq install ib_insync dataclasses
    import ib_insync as ibi
    ibi.util.startLoop()  # when in a notebook

     |████████████████████████████████| 72 kB 564 kB/s 
     |████████████████████████████████| 74 kB 2.8 MB/s 


In [ ]:
def install_ibg(force=False):
    if force or not os.path.exists('/opt/IBController/Scripts/DisplayBannerAndLaunch.sh'):
        root_dir = '/root'
        install_script = root_dir + '/install.sh'
        with open(install_script, "w") as f:
            f.write(inspect.cleandoc("""
                # Stop on error
                set -e
                # Pre-reqs
                apt-get install -qq -y wget unzip xvfb libxtst6 libxrender1 libxi6 x11vnc software-properties-common dos2unix net-tools >/dev/null
                # Download IB Gateway
                rm -rf /opt/TWS
                mkdir -p /opt/TWS
                cd /opt/TWS
                # wget -q http://cdn.quantconnect.com/interactive/ibgateway-latest-standalone-linux-x64-v974.4g.sh
                # chmod a+x ibgateway-latest-standalone-linux-x64-v974.4g.sh
                # mv ibgateway-latest-standalone-linux-x64-v974.4g.sh ibgateway-stable-standalone-linux-x64.sh
                wget -q https://download2.interactivebrokers.com/installers/ibgateway/stable-standalone/ibgateway-stable-standalone-linux-x64.sh
                chmod a+x ibgateway-stable-standalone-linux-x64.sh
                
                # Download / unpack IB Controller
                rm -rf /opt/IBController
                mkdir -p /opt/IBController/Logs
                cd /opt/IBController
                wget -q http://cdn.quantconnect.com/interactive/IBController-QuantConnect-3.2.0.5.zip
                unzip -qq ./IBController-QuantConnect-3.2.0.5.zip
                chmod -R u+x *.sh
                chmod -R u+x Scripts/*.sh
                echo "Made it this far 1"
                # Install IB Gateway
                /opt/TWS/ibgateway-stable-standalone-linux-x64.sh
                echo "Made it this far 2"
                # pull files from repo
                cd ~
                git clone https://github.com/mvberg/ib-gateway-docker
                cp ~/ib-gateway-docker/vnc/xvfb_init /etc/init.d/xvfb
                cp ~/ib-gateway-docker/vnc/vnc_init /etc/init.d/vnc
                cp ~/ib-gateway-docker/vnc/xvfb-daemon-run /usr/bin/xvfb-daemon-run
                dos2unix /usr/bin/xvfb-daemon-run /etc/init.d/xvfb /etc/init.d/vnc
                chmod -R 777 /usr/bin/xvfb-daemon-run /etc/init.d/xvfb /etc/init.d/vnc
                mkdir -p /root/IBController
                cp ~/ib-gateway-docker/ib/IBController.ini /root/IBController/IBController.ini
                cp ~/ib-gateway-docker/ib/jts.ini /root/Jts/jts.ini
                # Set US/Eastern timezone for IB Gateway - as default in that repo will override local TZ
                # sed -i 's%^TimeZone=.*$%TimeZone=US/Eastern%' /root/Jts/jts.ini
                sed -i 's%^TimeZone=.*$%TimeZone=UTC%' /root/Jts/jts.ini
            """))
        !chmod +x $install_script && $install_script
    else:
        print('IB Gateway is already installed')

def run_ibg(user, pwd, vnc_pwd, mode='paper', version='978', tz='UTC'):
    # if is_port_in_use(4001):
    #     print("Port 4001 already listens - IB Gateway ust be running...")
    #     return
    install_ibg()
    # TZ=America/Chicago
    env_vars = dict(
        TZ=tz,
        VNC_PASSWORD=vnc_pwd,
        TWS_MAJOR_VRSN=version,
        IBC_INI='/root/IBController/IBController.ini',
        IBC_PATH='/opt/IBController',
        TWS_PATH='/root/Jts',
        TWS_CONFIG_PATH='/root/Jts',
        LOG_PATH='/opt/IBController/Logs',
        JAVA_PATH='/opt/i4j_jres/1.8.0_152-tzdata2019c/bin',
        TRADING_MODE=mode,
        TWSUSERID=user,
        TWSPASSWORD=pwd,
        # FIXUSERID='',
        # FIXPASSWORD='',
        APP='GATEWAY',
        DISPLAY=':0',
    )
    for k, v in env_vars.items():
        os.environ[k] = v
    !nohup xvfb-daemon-run /opt/IBController/Scripts/DisplayBannerAndLaunch.sh &
    print('*** Checking start script output... ***')
    time.sleep(5)
    !cat nohup.out
    print('*** Sleeping 20 secs and checking log file... ***')
    time.sleep(20)
    !tail $(find $LOG_PATH -maxdepth 1 -type f -printf "%T@ %p\n" | sort -n | tail -n 1 | cut -d' ' -f 2-)
    print('*** Checking port 4001... ***')
    !netstat -anp | grep tcp | grep LISTEN | grep ':4001 '

run_ibg('gbroughton1234', 'A)$e3Mu84KYR', 'vncpwd')

Streaming output truncated to the last 5000 lines.
y

is invalid.
Where should IB Gateway 10.19 be installed?
[/root/Jts/ibgateway/1019]
The directory:

y

is invalid.
Where should IB Gateway 10.19 be installed?
[/root/Jts/ibgateway/1019]
The directory:

y

is invalid.
Where should IB Gateway 10.19 be installed?
[/root/Jts/ibgateway/1019]
The directory:

y

is invalid.
Where should IB Gateway 10.19 be installed?
[/root/Jts/ibgateway/1019]
The directory:

y

is invalid.
Where should IB Gateway 10.19 be installed?
[/root/Jts/ibgateway/1019]
The directory:

y

is invalid.
Where should IB Gateway 10.19 be installed?
[/root/Jts/ibgateway/1019]
The directory:

y

is invalid.
Where should IB Gateway 10.19 be installed?
[/root/Jts/ibgateway/1019]
The directory:

y

is invalid.
Where should IB Gateway 10.19 be installed?
[/root/Jts/ibgateway/1019]
The directory:

y

is invalid.
Where should IB Gateway 10.19 be installed?
[/root/Jts/ibgateway/1019]
The directory:

y

is invalid.
Where should IB 

KeyboardInterrupt: ignored

In [ ]:
# This should be in a separate cell and have a manual pause - it fails for me if I do like "Run All"
ib = ibi.IB()
ib.connect('127.0.0.1', 4001, clientId=44)

In [ ]:
!/opt/TWS/ibgateway-stable-standalone-linux-x64.sh

Unpacking JRE ...
Starting Installer ...
An error occurred:
java.lang.NoClassDefFoundError: Could not initialize class sun.awt.X11GraphicsEnvironment
Error log: /tmp/install4jError4054340271107456060.log
java.lang.NoClassDefFoundError: Could not initialize class sun.awt.X11GraphicsEnvironment
	at java.lang.Class.forName0(Native Method)
	at java.lang.Class.forName(Class.java:264)
	at java.awt.GraphicsEnvironment.createGE(GraphicsEnvironment.java:103)
	at java.awt.GraphicsEnvironment.getLocalGraphicsEnvironment(GraphicsEnvironment.java:82)
	at sun.awt.X11.XToolkit.<clinit>(XToolkit.java:132)
	at java.lang.Class.forName0(Native Method)
	at java.lang.Class.forName(Class.java:264)
	at java.awt.Toolkit$2.run(Toolkit.java:860)
	at java.awt.Toolkit$2.run(Toolkit.java:855)
	at java.security.AccessController.doPrivileged(Native Method)
	at java.awt.Toolkit.getDefaultToolkit(Toolkit.java:854)
	at java.awt.Toolkit.getEventQueue(Toolkit.java:1736)
	at java.awt.EventQueue.isDispatchThread(EventQueue